In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [12]:
df_data_pt_1 = pd.read_csv("Nomis Solutions_Data Part I.csv")
df_data_pt_2 = pd.read_csv("Nomis Solutions Part II.csv",skiprows=1)

In [13]:
df_data_pt_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208088 entries, 0 to 208087
Data columns (total 12 columns):
﻿Tier               208085 non-null float64
FICO                208087 non-null float64
Approve Date        208085 non-null object
Term                208085 non-null float64
Amount              208085 non-null float64
Previous Rate       208077 non-null object
Car  Type           208085 non-null object
Competition rate    208085 non-null float64
Outcome             208085 non-null float64
Rate                208085 non-null float64
Cost of Funds       208085 non-null float64
Partner Bin         208085 non-null float64
dtypes: float64(9), object(3)
memory usage: 19.1+ MB


In [15]:
 df_data_pt_2.head()

,Tier,FICO,Approve Date,Term,Amount,Previous Rate,Car Type,Competition Rate,Outcome,Rate,Cost of Funds,Partner Bin,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,2,702,7/2/02,60,22000,,U,5.85,0,6.19,1.84,3,NaN,Term,60,NaN,NaN,NaN
1,2,710,7/3/02,60,21000,,U,5.85,0,6.19,1.84,1,NaN,Car Type,U,NaN,NaN,NaN
2,3,693,7/6/02,60,19598,,U,5.85,1,7.29,1.84,1,NaN,Min FICO,684,NaN,NaN,NaN
3,3,696,7/6/02,60,23071,,U,5.85,0,7.29,1.84,3,NaN,Max FICO,712,NaN,NaN,NaN
4,3,697,7/8/02,60,21578,,U,5.80,1,7.29,1.84,2,NaN,Min Amnt,17800,NaN,NaN,NaN
